In [ ]:
# installing hugging face 'dataset' library
!pip install datasets


In [ ]:
# importing hugging face's dataset library
from datasets import load_dataset


In [ ]:
# load xsum dataset from hugging face
dataset = load_dataset("xsum")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# accessing train validation and test splits of the dataset
train_dataset = dataset["train"].select(range(100000))
# valid_dataset = dataset["validation"].select(range(1000))
# test_dataset = dataset["test"].select(range(1000))


In [ ]:
train_dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 100000
})

In [ ]:
import re

def remove_links_and_emails(example):
    # Define regex patterns for links and emails
    link_pattern = r'https?://\S+|www\.\S+'
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    # Compile regex for efficiency if applying this to many documents
    link_regex = re.compile(link_pattern)
    email_regex = re.compile(email_pattern)

    # Remove links and emails from 'document'
    example['document'] = link_regex.sub('', example['document'])
    example['document'] = email_regex.sub('', example['document'])

    # Remove links and emails from 'summary'
    example['summary'] = link_regex.sub('', example['summary'])
    example['summary'] = email_regex.sub('', example['summary'])

    return example

In [ ]:
for i, example in enumerate(train_dataset):
    if i < 10:
        print(example)
    else:
        break

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [ ]:
train_dataset_no_email_and_links = train_dataset.map(remove_links_and_emails)
# valid_dataset_no_email_and_links = valid_dataset.map(remove_links_and_emails)
# test_dataset_no_email_and_links = test_dataset.map(remove_links_and_emails)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def sent_tokenize_example(example):
    example['document'] = sent_tokenize(example['document'])
    example['summary'] = sent_tokenize(example['summary'])
    return example

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_sent_tokens = train_dataset_no_email_and_links.map(sent_tokenize_example)
# valid_sent_tokens = valid_dataset_no_email_and_links.map(sent_tokenize_example)
# test_sent_tokens = test_dataset_no_email_and_links.map(sent_tokenize_example)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': ['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.', 'Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.', 'Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.', 'Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.', 'First Minister Nicola Sturgeon visited the area to inspect the damage.', 'The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.', 'Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.', 'However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.', '"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I

In [ ]:
import string
string.punctuation
def remove_punctuations(example):
    table = str.maketrans('', '', string.punctuation)
    example['document'] = [word.translate(table) for word in example['document'] if word not in string.punctuation]
    example['summary'] = [word.translate(table) for word in example['summary'] if word not in string.punctuation]

    # Remove empty strings that may result from removing punctuation
    example['document'] = [token for token in example['document'] if token]
    example['summary'] = [token for token in example['summary'] if token]
    return example

In [ ]:
train_sent_tokens = train_sent_tokens.map(remove_punctuations)
# valid_sent_tokens = valid_sent_tokens.map(remove_punctuations)
# test_sent_tokens = test_sent_tokens.map(remove_punctuations)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': ['The full cost of damage in Newton Stewart one of the areas worst affected is still being assessed', 'Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water', 'Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct', 'Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town', 'First Minister Nicola Sturgeon visited the area to inspect the damage', 'The waters breached a retaining wall flooding many commercial properties on Victoria Street  the main shopping thoroughfare', 'Jeanette Tate who owns the Cinnamon Cafe which was badly affected said she could not fault the multiagency response once the flood hit', 'However she said more preventative work could have been carried out to ensure the retaining wall did not fail', 'It is difficult but I do think there is so much publicity for Dumfries and the Nith  and I totally appreciat

In [ ]:
#word tokenization
from nltk.tokenize import word_tokenize

def word_tokenize_sent_tokens(example):
    example['document']=[word_tokenize(sentence) for sentence in example['document']]
    example['summary']=[word_tokenize(sentence) for sentence in example['summary']]
    return example

In [ ]:
train_sent_word_tokens = train_sent_tokens.map(word_tokenize_sent_tokens)
# valid_sent_word_tokens = valid_sent_tokens.map(word_tokenize_sent_tokens)
# test_sent_word_tokens = test_sent_tokens.map(word_tokenize_sent_tokens)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_word_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': [['The', 'full', 'cost', 'of', 'damage', 'in', 'Newton', 'Stewart', 'one', 'of', 'the', 'areas', 'worst', 'affected', 'is', 'still', 'being', 'assessed'], ['Repair', 'work', 'is', 'ongoing', 'in', 'Hawick', 'and', 'many', 'roads', 'in', 'Peeblesshire', 'remain', 'badly', 'affected', 'by', 'standing', 'water'], ['Trains', 'on', 'the', 'west', 'coast', 'mainline', 'face', 'disruption', 'due', 'to', 'damage', 'at', 'the', 'Lamington', 'Viaduct'], ['Many', 'businesses', 'and', 'householders', 'were', 'affected', 'by', 'flooding', 'in', 'Newton', 'Stewart', 'after', 'the', 'River', 'Cree', 'overflowed', 'into', 'the', 'town'], ['First', 'Minister', 'Nicola', 'Sturgeon', 'visited', 'the', 'area', 'to', 'inspect', 'the', 'damage'], ['The', 'waters', 'breached', 'a', 'retaining', 'wall', 'flooding', 'many', 'commercial', 'properties', 'on', 'Victoria', 'Street', 'the', 'main', 'shopping', 'thoroughfare'], ['Jeanette', 'Tate', 'who', 'owns', 'the', 'Cinnamon', 'Cafe', 'which', 'was

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

# Function to convert NLTK's POS tags to the format expected by the WordNet lemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def pos_tag_and_lemmatize_example(example):
    temp = {'document':[],'summary':[],'id':''}
    temp['id']=example['id']
    for sentence in example['document']:
      tagged_tokens = pos_tag(sentence)
      temp['document'].append([lemmatizer.lemmatize(wordl, tagl) for wordl, tagl in [(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]])

    for sentence in example['summary']:
      tagged_tokens = pos_tag(sentence)
      temp['summary'].append([lemmatizer.lemmatize(wordl, tagl) for wordl, tagl in [(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]])
    return temp

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
train_sent_word_tokens = train_sent_word_tokens.map(pos_tag_and_lemmatize_example)
# valid_sent_word_tokens = valid_sent_word_tokens.map(pos_tag_and_lemmatize_example)
# test_sent_word_tokens = test_sent_word_tokens.map(pos_tag_and_lemmatize_example)

Parameter 'function'=<function pos_tag_and_lemmatize_example at 0x7fae5ed3de10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_word_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': [['The', 'full', 'cost', 'of', 'damage', 'in', 'Newton', 'Stewart', 'one', 'of', 'the', 'area', 'worst', 'affect', 'be', 'still', 'be', 'assess'], ['Repair', 'work', 'be', 'ongoing', 'in', 'Hawick', 'and', 'many', 'road', 'in', 'Peeblesshire', 'remain', 'badly', 'affect', 'by', 'stand', 'water'], ['Trains', 'on', 'the', 'west', 'coast', 'mainline', 'face', 'disruption', 'due', 'to', 'damage', 'at', 'the', 'Lamington', 'Viaduct'], ['Many', 'business', 'and', 'householder', 'be', 'affect', 'by', 'flood', 'in', 'Newton', 'Stewart', 'after', 'the', 'River', 'Cree', 'overflow', 'into', 'the', 'town'], ['First', 'Minister', 'Nicola', 'Sturgeon', 'visit', 'the', 'area', 'to', 'inspect', 'the', 'damage'], ['The', 'water', 'breach', 'a', 'retain', 'wall', 'flood', 'many', 'commercial', 'property', 'on', 'Victoria', 'Street', 'the', 'main', 'shopping', 'thoroughfare'], ['Jeanette', 'Tate', 'who', 'own', 'the', 'Cinnamon', 'Cafe', 'which', 'be', 'badly', 'affect', 'say', 'she', 'coul

In [ ]:
def lower(example):
    temp = {'document':[],'summary':[],'id':''}
    temp['id']=example['id']
    for sentence in example['document']:
      temp['document'].append([word.lower() for word in sentence])

    for sentence in example['summary']:
      temp['summary'].append([word.lower() for word in sentence])
    return temp

In [ ]:
train_sent_word_tokens = train_sent_word_tokens.map(lower)
# valid_sent_word_tokens = valid_sent_word_tokens.map(lower)
# test_sent_word_tokens = test_sent_word_tokens.map(lower)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_word_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': [['the', 'full', 'cost', 'of', 'damage', 'in', 'newton', 'stewart', 'one', 'of', 'the', 'area', 'worst', 'affect', 'be', 'still', 'be', 'assess'], ['repair', 'work', 'be', 'ongoing', 'in', 'hawick', 'and', 'many', 'road', 'in', 'peeblesshire', 'remain', 'badly', 'affect', 'by', 'stand', 'water'], ['trains', 'on', 'the', 'west', 'coast', 'mainline', 'face', 'disruption', 'due', 'to', 'damage', 'at', 'the', 'lamington', 'viaduct'], ['many', 'business', 'and', 'householder', 'be', 'affect', 'by', 'flood', 'in', 'newton', 'stewart', 'after', 'the', 'river', 'cree', 'overflow', 'into', 'the', 'town'], ['first', 'minister', 'nicola', 'sturgeon', 'visit', 'the', 'area', 'to', 'inspect', 'the', 'damage'], ['the', 'water', 'breach', 'a', 'retain', 'wall', 'flood', 'many', 'commercial', 'property', 'on', 'victoria', 'street', 'the', 'main', 'shopping', 'thoroughfare'], ['jeanette', 'tate', 'who', 'own', 'the', 'cinnamon', 'cafe', 'which', 'be', 'badly', 'affect', 'say', 'she', 'coul

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')  # Download the list of stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(example):
    temp = {'document':[],'summary':[],'id':''}
    temp['id']=example['id']
    for sentence in example['document']:
      temp['document'].append([word for word in sentence if word not in stop_words])

    for sentence in example['summary']:
      temp['summary'].append([word for word in sentence if word not in stop_words])
    return temp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_sent_word_tokens = train_sent_word_tokens.map(remove_stopwords)
# valid_sent_word_tokens = valid_sent_word_tokens.map(remove_stopwords)
# test_sent_word_tokens = test_sent_word_tokens.map(remove_stopwords)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_word_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': [['full', 'cost', 'damage', 'newton', 'stewart', 'one', 'area', 'worst', 'affect', 'still', 'assess'], ['repair', 'work', 'ongoing', 'hawick', 'many', 'road', 'peeblesshire', 'remain', 'badly', 'affect', 'stand', 'water'], ['trains', 'west', 'coast', 'mainline', 'face', 'disruption', 'due', 'damage', 'lamington', 'viaduct'], ['many', 'business', 'householder', 'affect', 'flood', 'newton', 'stewart', 'river', 'cree', 'overflow', 'town'], ['first', 'minister', 'nicola', 'sturgeon', 'visit', 'area', 'inspect', 'damage'], ['water', 'breach', 'retain', 'wall', 'flood', 'many', 'commercial', 'property', 'victoria', 'street', 'main', 'shopping', 'thoroughfare'], ['jeanette', 'tate', 'cinnamon', 'cafe', 'badly', 'affect', 'say', 'could', 'fault', 'multiagency', 'response', 'flood', 'hit'], ['however', 'say', 'preventative', 'work', 'could', 'carry', 'ensure', 'retain', 'wall', 'fail'], ['difficult', 'think', 'much', 'publicity', 'dumfries', 'nith', 'totally', 'appreciate', 'almost

In [ ]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

def summarize_by_page_rank(sentences):
  # Preprocess: Convert tokenized sentences to strings
  detokenizer = TreebankWordDetokenizer()
  sentences = [detokenizer.detokenize(sentence) for sentence in sentences]

  # Vectorize the sentences using TF-IDF
  vectorizer = TfidfVectorizer()
  sentence_vectors = vectorizer.fit_transform(sentences).toarray()

  # Compute similarity matrix
  similarity_matrix = cosine_similarity(sentence_vectors)

  # Build the graph
  nx_graph = nx.from_numpy_array(similarity_matrix)
  # Apply PageRank
  scores = nx.pagerank(nx_graph)

  # Rank sentences based on scores
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

  # Extract the top N sentences as the summary
  N = 1  # Number of sentences in the summary
  summary = ' '.join([ranked_sentences[i][1] for i in range(N)])
  return summary

In [ ]:
text = train_sent_word_tokens['document'][0] #sentence tokenized text
summarize_by_page_rank(text) #returns summary as per page rank algorithm using cosine similarity of tf idf

'affect flood dumfries galloway borders say important immediate step take protect area vulnerable clear timetable put place flood prevention plan'